In [22]:
# Importing libraries...
import pandas as pd
import matplotlib.pyplot as plt
!matplotlib inline
import seaborn as sns; sns.set()
import zipfile
from bs4 import BeautifulSoup
import os

'matplotlib' is not recognized as an internal or external command,
operable program or batch file.


In [6]:
df = pd.read_csv('data/bestofrt.tsv', sep='\t')

In [8]:
df.head()

,ranking,critic_score,title,number_of_critic_ratings
0,1,99,The Wizard of Oz (1939),110
1,2,100,Citizen Kane (1941),75
2,3,100,The Third Man (1949),77
3,4,99,Get Out (2017),282
4,5,97,Mad Max: Fury Road (2015),370


In [11]:
with zipfile.ZipFile('data/rt-html.zip', 'r') as myzip:
    myzip.extractall()

In [24]:
# List of dictionaries to build file by file and later convert to a DataFrame
df_list = []
folder = 'data/rt_html'

for movie_html in os.listdir(folder):
    with open(os.path.join(folder, movie_html)) as file:
        soup = BeautifulSoup(file, 'lxml')
        title = soup.find('title').contents[0][:-len(' - Rotten Tomatoes')]
        audience_score = soup.find('div', class_='audience-score meter').find('span').contents[0][:-1]
        num_audience_ratings = soup.find('div', class_='audience-info hidden-xs superPageFontColor')
        num_audience_ratings = num_audience_ratings.find_all('div')[1].contents[2].strip().replace(',', '')
        
        # Append to list of dictionaries
        df_list.append({'title': title,
                        'audience_score': int(audience_score),
                        'number_of_audience_ratings': int(num_audience_ratings)})
df = pd.DataFrame(df_list, columns = ['title', 'audience_score', 'number_of_audience_ratings'])

In [27]:
df.head()

,title,audience_score,number_of_audience_ratings
0,12 Angry Men (Twelve Angry Men) (1957),97,103672
1,The 39 Steps (1935),86,23647
2,The Adventures of Robin Hood (1938),89,33584
3,All About Eve (1950),94,44564
4,All Quiet on the Western Front (1930),89,17768
